<a href="https://colab.research.google.com/github/PuiPu/data_sci_final_project/blob/main/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ARIMA first test (Gemini)

In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Download the Apple stock prices data
df = pd.read_csv('/kaggle/input/apple-inc-aapl-stock-data-1980-2024/aapl_us_d.csv')

# Set the date as the index
df.set_index('Date', inplace=True)

# Convert the 'Close' column to numeric (assuming it's a string)
df['Close'] = df['Close'].str.replace(',', '').astype(float)

# Split the data into training and testing sets
train_data = df[:'2014-11-14']
test_data = df['2023-11-15':] # 先用到2023，照理講應該要到2024(總共14年)

# Build the ARIMA model
model = ARIMA(train_data['Close'], order=(2, 1, 1))  # Adjust the order parameters as needed
model_fit = model.fit()

# Forecast future prices
forecast = model_fit.forecast(steps=10)  # Forecast for the next 10 days

# Print the forecasted prices
print(forecast)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/apple-inc-aapl-stock-data-1980-2024/aapl_us_d.csv'